## Weekly Review (2022. 09. 13 ~ 16.)
* TUE : Matplotlib, Titanic dataset Analysis
* WED : Github Update, Matplotlib, Titanic dataset Analysis
* THU : Folium, Market dataset
* FRI : Folium, Pandas, Titanic dataset Analysis

### Titanic Dataset 타이타닉 생존자 예측 대회
#### 학습목표
* 캐글 대회에 참여해 봅니다.

#### 학습내용
    1-1 캐글 대회 입문하기
    1-2 데이터 불러오기
    1-3 데이터 탐색하기
    1-4 모델(knn) 만들고 제출해보기

#### 준비
* 대회 링크 : https://www.kaggle.com/c/titanic

### Data 확인하기
* survival : 생존여부 (0 = No, 1 = Yes)
* Pclass : 티켓의 클래스 (Ticket class, 1 = 1st, 2 = 2nd, 3 = 3rd)
* Sex : 성별 (male / female)
* Age : 나이
* SibSp : 함께 탑승한 형제와 배우자의 수 (Silbings, spouses aboard the Titanic.)
  * Sibings : 형제, 자매, 의붓 형제
  * Spouses : 남편, 아내 (정부와 약혼자는 무시)
* Parch : 함께 탑승한 부모, 아이의 수
  * Parch : Parent(mother, father), child(daughter, son, stepdaughter, stepson)
* Ticket : 티켓 번호 (ex. CA 31352, A/5.2151)
* Fare : 탐승료
* Cabin : 객실 번호
* Embarked : 탑승 항구 (C = Cherbourg, Q = Queenstown, S = Southampton)

### 01. 데이터 셋 불러오기
* train.csv : 학습을 위한 데이터 셋
* test.csv : 테스트를 위한 데이터 셋
* gender_submission.csv : 예측 내용 제출 데이터 셋

In [2]:
import pandas as pd

In [4]:
train = pd.read_csv("data/titanic/train.csv")
test = pd.read_csv("data/titanic/test.csv")
sub = pd.read_csv("data/titanic/gender_submission.csv")

### 02. 데이터 탐색하기
* 데이터의 행과 열(shape)
* column명 확인하기
* 각 컬럼의 자료형 확인하기
* 컬럼 결측치 확인하기
* 앞/뒤 데이터 확인하기
* 각 열 요약값 확인하기

#### 데이터의 행과 열

In [6]:
train.shape, test.shape, sub.shape

((891, 12), (418, 11), (418, 2))

#### 컬럼명 확인하기

In [8]:
print(train.columns)
print(test.columns)

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')
Index(['PassengerId', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp', 'Parch',
       'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')


#### 각 컬럼의 자료형 확인하기

In [9]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


* Age와 Cabin과 Embarked에 결측치가 있다.

In [10]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  418 non-null    int64  
 1   Pclass       418 non-null    int64  
 2   Name         418 non-null    object 
 3   Sex          418 non-null    object 
 4   Age          332 non-null    float64
 5   SibSp        418 non-null    int64  
 6   Parch        418 non-null    int64  
 7   Ticket       418 non-null    object 
 8   Fare         417 non-null    float64
 9   Cabin        91 non-null     object 
 10  Embarked     418 non-null    object 
dtypes: float64(2), int64(4), object(5)
memory usage: 36.0+ KB


* Age와 Fare와 Cabin에 결측치가 있다.

In [11]:
train.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [12]:
test.isnull().sum()

PassengerId      0
Pclass           0
Name             0
Sex              0
Age             86
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
dtype: int64

#### 데이터의 앞/뒤 확인하기

In [13]:
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [14]:
train.tail()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.00,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.00,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.45,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.00,C148,C
890,891,0,3,"Dooley, Mr. Patrick",male,32.0,0,0,370376,7.75,NaN,Q


In [15]:
test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [16]:
test.tail()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
413,1305,3,"Spector, Mr. Woolf",male,NaN,0,0,A.5. 3236,8.0500,NaN,S
414,1306,1,"Oliva y Ocana, Dona. Fermina",female,39.0,0,0,PC 17758,108.9000,C105,C
415,1307,3,"Saether, Mr. Simon Sivertsen",male,38.5,0,0,SOTON/O.Q. 3101262,7.2500,NaN,S
416,1308,3,"Ware, Mr. Frederick",male,NaN,0,0,359309,8.0500,NaN,S
417,1309,3,"Peter, Master. Michael J",male,NaN,1,1,2668,22.3583,NaN,C


#### 각 열 요약값 확인하기

In [17]:
train.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [18]:
test.describe()

,PassengerId,Pclass,Age,SibSp,Parch,Fare
count,418.000000,418.000000,332.000000,418.000000,418.000000,417.000000
mean,1100.500000,2.265550,30.272590,0.447368,0.392344,35.627188
std,120.810458,0.841838,14.181209,0.896760,0.981429,55.907576
min,892.000000,1.000000,0.170000,0.000000,0.000000,0.000000
25%,996.250000,1.000000,21.000000,0.000000,0.000000,7.895800
50%,1100.500000,3.000000,27.000000,0.000000,0.000000,14.454200
75%,1204.750000,3.000000,39.000000,1.000000,0.000000,31.500000
max,1309.000000,3.000000,76.000000,8.000000,9.000000,512.329200


In [20]:
train.describe(include=['O'])

,Name,Sex,Ticket,Cabin,Embarked
count,891,891,891,204,889
unique,891,2,681,147,3
top,"Hakkarainen, Mr. Pekka Pietari",male,347082,B96 B98,S
freq,1,577,7,4,644


In [21]:
test.describe(include=['O'])

,Name,Sex,Ticket,Cabin,Embarked
count,418,418,418,91,418
unique,418,2,363,76,3
top,"Kreuchen, Miss. Emilie",male,PC 17608,B57 B59 B63 B66,S
freq,1,266,5,3,270


### 03. 모델 만들고 제출해보기
* 학습용 데이터 셋에서 학습할 컬럼을 선택한다.
* 모델 선택 후 학습한다.
  * 모델(knn)을 생성 후 학습(fit)
* 예측(predict)를 수행 후 제출한다.

#### 라이브러리 불러오기

In [22]:
from sklearn.neighbors import KNeighborsClassifier

#### 데이터 준비 : 빠른 모델 생성을 위해 일단은 전처리가 불필요한 featuer(변수)만 선택
* 'Survived'를 제외 (예측해야 할 feature)
* 'Embarked', 'Sex', 'Name', 'Ticket' -> 문자 포함
* 'Age' : 결측치가 있음

In [23]:
sel = ['PassengerId', 'Pclass', 'SibSp', 'Parch']

#### 학습에 사용될 데이터 준비

In [24]:
X_train = train[sel]
y_train = train['Survived']

#### 학습 후 테스트에 사용될 피처 선택

In [25]:
X_test = test[sel]

X_train.shape, y_train.shape, X_test.shape

((891, 4), (891,), (418, 4))

#### 모델 선택

In [26]:
model = KNeighborsClassifier()

#### 학습

In [27]:
model.fit(X_train, y_train)

KNeighborsClassifier()

#### 예측

In [28]:
pred = model.predict(X_test)
pred[:15]  # 예측한 값을 15개 정도 확인

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int64)

#### 제출

In [29]:
sub.columns

Index(['PassengerId', 'Survived'], dtype='object')

In [30]:
sub['Survived'] = pred
sub.to_csv("first_sub.csv", index=False)

### 04. 제출 절차
* Titanic 대회 이동
* 'Leaderboard' 선택
   * https://www.kaggle.com/competitions/titanic/leaderboard
* 'Submit Predictions' 선택
* 제출용 파일을 Upload을 수행.
* 간단한 설명이 필요하면 이를 쓰고, 'Make Submission'을 선택하여 제출 수행.
* 제출이 정상적으로 이루어지면 나의 현재 스코어와 Rank의 확인이 가능하다.

#### (실습) 변수를 2개만 선택했다 PassengerID, Parch를 각각 하나씩 추가해 가면서 모델 만들고 제출해보기.
#### (도전) Fare, Age의 결측값을 채울 수 있다. 이를 채워서 모델의 입력으로 넣어서 모델을 만들고 제출해보자.